<a href="https://colab.research.google.com/github/lynnseygraham/cs_assignment2/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:

#Assign values of known variables:
e = 5
n = 35
c = 10
d = 0
M = 0

#Find two primes, p and q, that multiply to equal n:
p_q = []
for i in range(2, n):
  if n % i == 0:
    p_q.append(i)

#Find Øn:
Øn = (p_q[0]-1) * (p_q[1]-1)

#Find d such that d*e = 1 % Øn and d < Øn:
for i in range(2, Øn):
  if i * e % Øn == 1:
    d = i

#Find M by calculating c**d % n:
M = c**d % n

print('a) The value of plaintext M is:', M)
print('b) The value of the secret exponent d is:', d)

a) The value of plaintext M is: 5
b) The value of the secret exponent d is: 5


# Q2

In [ ]:
q = 11
a = 2
# Assign values of known private keys:
Y_A = 9
Y_B = 3
# Initialize variables for unknown private keys:
X_A = 0
X_B = 0

#a) What is A's private key, X_A, given that Y_A=9?
for i in range(1, 100):
  x = a**i % q
  if x == Y_A:
    X_A = i
    print('a) A\'s private key is:', X_A)
    break

#b) What is secret key K shared with A?
K = Y_B**X_A % q
print('b) The shared secret key is:', K)


a) A's private key is: 6
b) The shared secret key is: 3


In [129]:
def is_prime(n):
  if n == 2 or n == 3: return True
  if n < 2 or n%2 == 0: return False
  if n < 9: return True
  if n%3 == 0: return False
  r = int(n**0.5)
  f = 5
  while f <= r:
    if n % f == 0: return False
    if n % (f+2) == 0: return False
    f += 6
  return True


#Q3


In [150]:
#a) Compute hashes of 16 sets of 3 random prime numbers less than 2^8 (256)
# Generate set of all primes < 256

prime_list = []
comp_list = []
for i in range(1, 256):
  if i > 1:
    for j in range(2, i):
      if (i % j) == 0:
        comp_list.append(i)
      else:
        if is_prime(i):
          prime_list.append(i)
        #print(i)
#print(prime_list)
prime_set = list(set(prime_list))
#print(prime_set)
comp_set = list(set(comp_list))
#print(prime_set)
#print(comp_set)

# Generate 16 sets of 3 random primes from set generated above
import random
randset = []
xor_hashes = []
print('List of 16 Valid messages:')
for i in range(0,16):
  randset.append(random.sample(prime_set, k=3))
  xor_hashes.append(randset[i][0] ^ randset[i][1] ^ randset[i][2])
  print('{:<16} {:1}'.format(str(randset[i]), ' || '+str(xor_hashes[i])))

print('\nLooking for fraud message that matches valid message\'s hash...')

comp_randset = []
fraud_hashes = []
for i in range(0, 1000):
  comp_randset.append(random.sample(comp_list, k=3))
  fraud_hashes.append(comp_randset[i][0] ^ comp_randset[i][1] ^ comp_randset[i][2])
  if fraud_hashes[i] in xor_hashes:
    index = xor_hashes.index(fraud_hashes[i])
    print('\nCollision found after', i, 'attempts:')
    print('{:<16} {:1}'.format(str(randset[index]), ' || '+str(xor_hashes[index])))
    print('{:<16} {:1}'.format(str(comp_randset[i]), ' || '+str(fraud_hashes[i])))
    break


List of 16 Valid messages:
[47, 107, 31]     || 91
[53, 163, 7]      || 145
[41, 239, 223]    || 25
[199, 173, 127]   || 21
[71, 103, 53]     || 21
[37, 5, 53]       || 21
[47, 43, 29]      || 25
[199, 13, 241]    || 59
[83, 127, 107]    || 71
[163, 157, 83]    || 109
[127, 59, 83]     || 23
[163, 227, 229]   || 165
[41, 139, 31]     || 189
[131, 101, 43]    || 205
[229, 73, 197]    || 105
[29, 233, 53]     || 193

Looking for fraud message that matches valid message's hash...

Collision found after 3 attempts:
[53, 163, 7]      || 145
[244, 108, 9]     || 145


#Q4

In [ ]:
!pip install cryptography
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

     |████████████████████████████████| 2.6MB 2.7MB/s 


In [131]:
def main():
  for i in range(0, 10000000):
    prime_list = []
    comp_list = []
    for i in range(1, 256):
      if i > 1:
        for j in range(2, i):
          if (i % j) == 0:
            comp_list.append(i)
          else:
            if is_prime(i):
              prime_list.append(i)
            #print(i)
    prime_set = list(set(prime_list))
    comp_set = list(set(comp_list))
    #print(prime_set)
    #print(comp_set)

    import random
    randset = []
    sha_hashes = []
    print('List of 16 Valid messages:')
    for i in range(0,16):
      randset.append(random.sample(prime_set, k=3))
      digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
      digest.update(str(randset[i][0]).encode())
      digest.update(str(randset[i][1]).encode())
      digest.update(str(randset[i][2]).encode())
      sha_hashes.append(digest.finalize())
      print('{:<16s} {:1}'.format(str(randset[i]), ' || '+str(sha_hashes[i])))

    print('\nFirst 10/1000 hashes generated:')

    print_count = 10
    comp_randset = []
    fraud_hashes = []
    cols_found = False
    for i in range(0, 1000):
      comp_randset.append(random.sample(comp_list, k=3))
      digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
      digest.update(str(comp_randset[i][0]).encode())
      digest.update(str(comp_randset[i][1]).encode())
      digest.update(str(comp_randset[i][2]).encode())
      fraud_hashes.append(digest.finalize())

      if print_count:
          print('{:<16s} {:1}'.format(str(comp_randset[i]), ' || '+str(fraud_hashes[i])))
          print_count -= 1

      if fraud_hashes[i] in sha_hashes:
        index = sha_hashes.index(fraud_hashes[i])
        print('\nCollision found after', i, 'attempts:')
        print('{0:10} {1}'.format(str(randset[index]), ' || '+str(sha_hashes[index])))
        print('{0:10} {1}'.format(str(comp_randset[i]), ' || '+str(fraud_hashes[i])))
        cols_found = True
        break

    print('\nLooking for fraud message that matches valid message\'s hash...')

    if not cols_found:
      print('No collisions were found.')
    else:
      break

main()

Streaming output truncated to the last 5000 lines.
[28, 135, 200]    || b'\xb3\xb3\xa8\xe0\x8d\xaf\xe6\xb8H\xf6(\xa2@B\x94\x07\xd4\xec\n\xcf15\xba\xa0t5\xa6\xa2d\x1c\x13*'

Looking for fraud message that matches valid message's hash...
No collisions were found.
List of 16 Valid messages:
[73, 43, 137]     || b'\xe8\x11\xc1\x919\x99\x95\xa3\xfd\x9e\x08/,\xd9=ds\x91=\xf4\x8e\xday\x84\xbft\xa1\x8c\x9fmEY'
[19, 211, 89]     || b'_L\xf6+\xeb\x95\xb7-\xa5j\xc6\xfb:\xe3(\x97u\xf1<\x91\xbf/\x8d\xb5\xdc\xb9\xe2f\xc2M\x002'
[173, 61, 211]    || b'2p\x88\xa9\xd9KE.\xf7=\xe3t\xbf\x13\xdaS\x83\xf0p\xfew\xbd\xffPL\x81?\x1bjavN'
[227, 67, 233]    || b'\xe7\xe3l\xcb\x97\x18_\xbc\x1a\x03\xcd\x7fc)GJ\xac\x9a\xf8\xab9\x87\xbb\x9d\x9a\xb0\x02Iu\xb0\xc5O'
[71, 43, 7]       || b"\xdb\xed\xdd\x893P\xd4\x8b\xff\xdf&'}\ny\xef|!Cm\xb3\xec\xb4\x1d/Z\xb2\xe4\xcaD\x8f\x10"
[251, 113, 67]    || b'Fzt\xda\xb6\x0f<\xc2\x1e\xd3p\x17;\x0fe\xa2\xc5>9G\x14\xbc\x9a=\x17\xa0\xf7\xb3~;\xeeg'
[151, 101, 109]   || b'\x19\xdd\

In [151]:
prime_set = [19, 223, 199]
comp_set = [192, 231, 99]

pr_digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
pr_digest.update(str(prime_set[0]).encode())
pr_digest.update(str(prime_set[1]).encode())
pr_digest.update(str(prime_set[2]).encode())
prime_hash = pr_digest.finalize()
print(prime_set, '|| ', prime_hash)

co_digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
co_digest.update(str(prime_set[0]).encode())
co_digest.update(str(prime_set[1]).encode())
co_digest.update(str(prime_set[2]).encode())
comp_hash = co_digest.finalize()
print(comp_set, ' || ', comp_hash)

[19, 223, 199] ||  b'\xfdJ*\x87\xb3b\x1d\x8f6\xab\xe8\x8e\xd5\xeb1O\xd1\xbaI\xb96\x00\xc2n\xd1O%\xc9\xaa\x9c\xce\xb9'
[192, 231, 99]  ||  b'\xfdJ*\x87\xb3b\x1d\x8f6\xab\xe8\x8e\xd5\xeb1O\xd1\xbaI\xb96\x00\xc2n\xd1O%\xc9\xaa\x9c\xce\xb9'
